# Collecting the post's content data

Using this code we can open a Realtor post and scrape data from that web page. After that we converted the collected data to dataframe and finally append it as a row to a CSV file.

In [1]:
from json.tool import main
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
import csv
from csv import writer
import pandas as pd
import os
import wget
import sys

Opening the file that contains posts' link:

In [2]:
# Open links file
df = pd. read_csv('Realtorlinks_2022-03-28.csv', header =None, usecols=[0])
df.head()

,0
0,url
1,https://www.realtor.ca/real-estate/24181975/10...
2,https://www.realtor.ca/real-estate/24181978/69...
3,https://www.realtor.ca/real-estate/24181886/10...
4,https://www.realtor.ca/real-estate/24181361/32...


Making a directory for downloading and saving images

In [ ]:
# Make directory for downloading and saving images
path2 = os.getcwd()
path2 = os.path.join(path2 , 'homeimages')
os.mkdir(path2)
path2

In [ ]:
# Just for manually setting start point of getting links
counter = 425
except_count = 0

Opening Chrome Driver

In [ ]:
# Opening Chrome Driver
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

The following part is the main body of the code. As you see we implemented a while loop that crwal of links and then open their URL. After opening each post, selected attributes been scraped and saved in a list. Then they save as a row in output file.

In [ ]:
# Main Body
while counter < len(df):
    url = df.iat[counter,0]
    print(counter)
    print(url)
    driver.get(url)
    try:
        element_present = EC.presence_of_element_located((By.ID, 'listingDetailsTopCon'))
        WebDriverWait(driver, 30).until(element_present)
        now = datetime.now() # current date and time
        scrapeDatetime = now.strftime("%m/%d/%Y, %H:%M:%S")
        #print (scrapeDatetime)
        listingDetailsTopCon = driver.find_element(by=By.ID, value="listingDetailsTopCon")
        ls = listingDetailsTopCon.text.split('\n')
        bedrooms = ''
        totalbedrooms = 0
        bathrooms = ''
        mls = ''
        price = ''
        addressline1 = ''
        addressline2 = ''
        po = ''
        for l in ls:
            if (l.find('$') != -1):
                i= ls.index(l)
                price = ls[i]
                addressline1 = ls[i+2]
            if (l.find('MLS') != -1):
                k = ls.index(l)
                mls = ls[k]
                addressline2 = ls[k-1]
                po = addressline2[-6:]
            if (l.find('Bedrooms') != -1):
                i = ls.index(l)
                bedrooms = ls[i-1]
                totalbedrooms = int(ls[i-1][0])+ int(ls[i-1][-1:])
            if (l.find('Bathrooms') != -1):
                i = ls.index(l)
                bathrooms = ls[i-1] 
        #print(ls)
        obj = [scrapeDatetime, url, price, addressline1, addressline2, po, mls, bedrooms, totalbedrooms, bathrooms]
        #print(obj)
        PropertySummary = driver.find_element(by=By.ID, value="PropertySummary")
        ps = PropertySummary.text.split('\n')
        #print(ps)
        propertyType = ''
        BuildingType = ''
        storeys = ''
        communityName = ''
        title = ''
        landSize = ''
        Builtin = ''
        parkingType = ''
        publishTime = ''
        for item in ps:
            if item == 'Property Type':
                k = ps.index(item)
                propertyType = ps[k+1]
            if item == 'Building Type':
                k = ps.index(item)
                BuildingType = ps[k+1]
            if item == 'Storeys':
                k = ps.index(item)
                storeys = ps[k+1]
            if item == 'Community Name':
                k = ps.index(item)
                communityName  = ps[k+1]
            if item == 'Title':
                k = ps.index(item)
                title = ps[k+1]
            if item == 'Land Size':
                k = ps.index(item)
                landSize = ps[k+1]
            if item == 'Built in':
                k = ps.index(item)
                Builtin = ps[k+1]
            if item == 'Parking Type':
                k = ps.index(item)
                parkingType = ps[k+1]
            if item == 'Time on REALTOR.ca':
                k = ps.index(item)
                publishTime = ps[k+1]
        obj.extend([propertyType, BuildingType, storeys, communityName, title, landSize, Builtin, parkingType, publishTime])
        #print(obj)
        try:
            PriceHistory = driver.find_element(by=By.ID, value="historyDetailSection")
            ph = PriceHistory.text.split('\n')
            priceHistory = ph[2]
        except:
            ph[2] = ''
        #print(ph[2])
        obj.append(ph[2])
        listingDetailsBuildingCon = driver.find_element(by=By.ID, value="listingDetailsBuildingCon")
        lsd = listingDetailsBuildingCon.text.split('\n')
        totalFinishedArea = ''
        appliancesIncluded = ''
        foundationType = ''
        style = ''
        architectureStyle = ''
        basementType = ''

        for item in lsd:
            if item == 'Total Finished Area':
                k = lsd.index(item)
                totalFinishedArea = lsd[k+1]
            if item == 'Appliances Included':
                k = lsd.index(item)
                appliancesIncluded = lsd[k+1]
            if item == 'Foundation Type':
                k = lsd.index(item)
                foundationType = lsd[k+1]
            if item == 'Style':
                k = lsd.index(item)
                style = lsd[k+1]
            if item == 'Architecture Style':
                k = lsd.index(item)
                architectureStyle = lsd[k+1]
            if item == 'Basement Type':
                k = lsd.index(item)
                basementType = lsd[k+1]
        obj.extend([totalFinishedArea, appliancesIncluded, foundationType, style, architectureStyle, basementType])
        obj.extend(lsd)
        images = driver.find_elements(by=By.ID, value='propimg_1')
        images = [image.get_attribute('src') for image in images]
        #print(images)
        for image in images:
            imglnk = image.split('/')
            file_name = imglnk[len(imglnk)-1]
            #print(file_name)
            save_as = os.path.join(path2,  file_name)
            wget.download(image, save_as)
            obj.extend([file_name])
        counter = counter + 1
        print(obj)
        # writing in the CSV file
        csv_name = "Realtor_RealEstatePage_JP_S2.csv"
        with open(csv_name, 'a', newline='') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(obj)
            f_object.close()
        except_count = 0
    except Exception as e:
        print('Error:', e.__class__)
        except_count = except_count + 1
        print("Try number:",except_count)
        if except_count >= 3:
            counter = counter + 1
            print("Trys exceeded! Go to next link...")

To close Chrome driver:

In [ ]:
# To close Chrome driver
driver.quit()